In [1]:
from importlib import reload
import matplotlib.pyplot as plt
from scipy import spatial
import numpy as np
from scipy.linalg import eig, eigh

import merger_tree_tools as mtt
import snapshot_tools as st
import halo_tools as ht
import galaxy_tools as gt
from time import sleep, perf_counter

from halomod import TracerHaloModel
import halomod
import hmf

In [2]:
mtt=reload(mtt)
st=reload(st)
ht=reload(ht)
# gt=reload(gt)


# snap=st.SnapshotTools('/fred/oz009/cpower/L31pt25_N128_2LPT/HiSURFS_L31pt25_0076/HiSURFS_L31pt25_0076','HDF5', convention='SWIFT',positions_only=False)
# halo=ht.HaloTools('/fred/oz009/cpower/L31pt25_N128_2LPT/halos/HiSURFS_L31pt25_0076.VELOCIraptor','VELOCIraptor')
# tree=mtt.TreeTools('/fred/oz009/cpower/L31pt25_N128_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')

# snap=st.SnapshotTools('/fred/oz009/cpower/L31pt25_N512_2LPT/HiSURFS_L31pt25_0076/HiSURFS_L31pt25_0076','HDF5', convention='SWIFT',positions_only=False)
# halo=ht.HaloTools('/fred/oz009/cpower/L31pt25_N512_2LPT/halos/HiSURFS_L31pt25_0076.VELOCIraptor','VELOCIraptor')
# tree=mtt.TreeTools('/fred/oz009/cpower/L31pt25_N512_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')

# snap.ReadSnapshot()
# halo.ReadHaloCatalogue()
# tree.ReadMergerTreeCatalogue()

# snap=st.SnapshotTools('/fred/oz009/cpower/L62pt5_N512_2LPT/HiSURFS_L62pt5_0076/HiSURFS_L62pt5_0076','HDF5', convention='SWIFT',positions_only=False)
# halo=ht.HaloTools('/fred/oz009/cpower/L62pt5_N512_2LPT/halos/HiSURFS_L62pt5_0076.VELOCIraptor','VELOCIraptor')
# tree=mtt.TreeTools('/fred/oz009/cpower/L62pt5_N512_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')

# snap.ReadSnapshot()
# halo.ReadHaloCatalogue()
# tree.ReadMergerTreeCatalogue()

# snap=st.SnapshotTools('/fred/oz009/cpower/L125_N512_2LPT/HiSURFS_L125_0076/HiSURFS_L125_0076','HDF5', convention='SWIFT',positions_only=False)
# halo=ht.HaloTools('/fred/oz009/cpower/L125_N512_2LPT/halos/HiSURFS_L125_0076.VELOCIraptor','VELOCIraptor')
# tree=mtt.TreeTools('/fred/oz009/cpower/L125_N512_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')

# snap.ReadSnapshot()
# halo.ReadHaloCatalogue()
# tree.ReadMergerTreeCatalogue()

snap=st.SnapshotTools('/fred/oz009/cpower/L250_N512_2LPT/HiSURFS_L250_0076/HiSURFS_L250_0076','HDF5', convention='SWIFT',positions_only=False)
halo=ht.HaloTools('/fred/oz009/cpower/L250_N512_2LPT/halos/HiSURFS_L250_0076.VELOCIraptor','VELOCIraptor')
tree=mtt.TreeTools('/fred/oz009/cpower/L250_N512_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')
snap.ReadSnapshot()
halo.ReadHaloCatalogue()
tree.ReadMergerTreeCatalogue()

# snap=st.SnapshotTools('/fred/oz009/cpower/L500_N512_2LPT/HiSURFS_L500_0076/HiSURFS_L500_0076','HDF5', convention='SWIFT',positions_only=False)
# halo=ht.HaloTools('/fred/oz009/cpower/L500_N512_2LPT/halos/HiSURFS_L500_0076.VELOCIraptor','VELOCIraptor')
# tree=mtt.TreeTools('/fred/oz009/cpower/L500_N512_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')
# snap.ReadSnapshot()
# halo.ReadHaloCatalogue()
# tree.ReadMergerTreeCatalogue()

# snap=st.SnapshotTools('/fred/oz009/cpower/L1000_N512_2LPT/HiSURFS_L1000_0076/HiSURFS_L1000_0076','HDF5', convention='SWIFT',positions_only=False)
# halo=ht.HaloTools('/fred/oz009/cpower/L1000_N512_2LPT/halos/HiSURFS_L1000_0076.VELOCIraptor','VELOCIraptor')
# tree=mtt.TreeTools('/fred/oz009/cpower/L1000_N512_2LPT/trees/VELOCIraptor.walkabletree.forestID.hdf5.0','TreeFrog')
# snap.ReadSnapshot()
# halo.ReadHaloCatalogue()
# # # gal.ReadGalaxyCatalogue()
# tree.ReadMergerTreeCatalogue()

Reading data from /fred/oz009/cpower/L250_N512_2LPT/HiSURFS_L250_0076/HiSURFS_L250_0076.hdf5
Simulation scale factor: 1.000000
Number of particles: 0134217728
Number of particle types: 7
Reading data for 158233 groups
['File_id', 'Group_Size', 'Num_of_files', 'Num_of_groups', 'Number_of_substructures_in_halo', 'Offset', 'Offset_unbound', 'Parent_halo_ID', 'Total_num_of_groups']
Reading data for 158233 groups
Reading data for 51865701 particles in groups
Reading data for 2332668 particles in groups
Found data for 077 snapshots in file...


In [3]:
final_snap_num=76
halo_list=np.logical_and(halo.Structuretype==10,halo.GroupMFOF>snap.Particle_mass*600)

main_subhalo_list=halo.GroupID[halo_list]
print("Tracking merger histories for %04d halos"%main_subhalo_list.size)

start_time=perf_counter()

kdtree=spatial.KDTree(snap.pos,compact_nodes=True, copy_data=False, balanced_tree=True,leafsize=10,boxsize=1.001*snap.BoxSize)

end_time=perf_counter()
print(f'It took {end_time-start_time:0.2f} seconds (s)')

Tracking merger histories for 11718 halos
It took 54.32 seconds (s)


In [4]:
start_time=perf_counter()

NumGroups=main_subhalo_list.size

formation_redshift=np.zeros(NumGroups)
alpha03=np.zeros(NumGroups)

Nsub=np.zeros(NumGroups)
HostID=np.zeros(NumGroups)
msub=np.zeros(NumGroups)
frac=np.zeros(NumGroups)

veldisp_3D_sub=np.zeros(NumGroups)
veldisp_1D_sub=np.zeros(NumGroups)

halo_masstot=np.zeros(NumGroups)
halo_m200=np.zeros(NumGroups)
halo_mvir=np.zeros(NumGroups)
halo_mfof=np.zeros(NumGroups)

offset=np.zeros(NumGroups)
number=np.zeros(NumGroups)
r200=np.zeros(NumGroups)
rvir=np.zeros(NumGroups)

Ekin=np.zeros(NumGroups)
Epot=np.zeros(NumGroups)
eta=np.zeros(NumGroups)
angular_momentum=np.zeros(NumGroups)
spin=np.zeros(NumGroups)

veldisp_3d=np.zeros(NumGroups)
veldisp_1d=np.zeros(NumGroups)

siga=np.zeros(NumGroups)
sigb=np.zeros(NumGroups)
sigc=np.zeros(NumGroups)
sigv=np.zeros(NumGroups)
c_a=np.zeros(NumGroups)
b_a=np.zeros(NumGroups)

particle_mass=np.zeros(NumGroups)
boxsize=np.zeros(NumGroups)

Nsub=np.zeros(NumGroups)
m200_sub=np.zeros(NumGroups)
mtot_sub=np.zeros(NumGroups)
mvir_sub=np.zeros(NumGroups)
mfof_sub=np.zeros(NumGroups)

veldisp_sub=np.zeros(NumGroups)
veldisp_submbp=np.zeros(NumGroups)
veldisp_subminpot=np.zeros(NumGroups)
Vmax=np.zeros(NumGroups)
Vmax_sub=np.zeros(NumGroups)
frac=np.zeros(NumGroups)
frac_v=np.zeros(NumGroups)
major_merger=np.zeros(NumGroups)
max_mass=np.zeros(NumGroups)
max_redshift=np.zeros(NumGroups)

def find_most_massive_merger(redshift, mass):
    sorted_indices = np.argsort(mass)[::-1]
    sorted_redshift = redshift[sorted_indices]
    sorted_mass = mass[sorted_indices]
    max_massive_diff = 0
    max_massive_redshift = None
    for i in range(len(sorted_mass) - 1):
        massive_diff = sorted_mass[i] - sorted_mass[i+1]
        if massive_diff > max_massive_diff:
            max_massive_diff = massive_diff
            max_massive_redshift = sorted_redshift[i]
    return max_massive_redshift

def find_max_mass_redshift(redshift, mass):
    valid_indices = redshift < 0.5
    valid_redshift = redshift[valid_indices]
    valid_mass = mass[valid_indices]
    
    sorted_indices = np.argsort(valid_mass)[::-1]
    sorted_mass = valid_mass[sorted_indices]
    
    max_mass = 0
    max_redshift = None
    
    for i in range(len(sorted_mass) - 1):
        gai = (sorted_mass[i] - sorted_mass[i+1]) / sorted_mass[i]
        if gai > max_mass:
            max_mass = gai
            max_redshift = valid_redshift[sorted_indices[i]]
    
    return max_mass, max_redshift


radius_list = [1.0, 2.5, 5.0]
rho_results = np.zeros((NumGroups,len(radius_list)))

G=43
ihalo=0
for main_halo in main_subhalo_list:
    halo_id=np.where(main_halo==halo.GroupID)[0]
    # print(halo_id, main_halo, final_snap_num)
    redshift,mass,m200,group_number,subhalo_number,zform,alpha=tree.TrackMainHaloProgenitor(main_halo,final_snap_num)
    
    if len(mass)>0:
        # assemble history
        formation_redshift[ihalo]=zform
        alpha03[ihalo]=alpha

        # general properties
        halo_masstot[ihalo]=mass[0]
        halo_m200[ihalo]=m200[0]
        halo_mvir[ihalo]=halo.GroupMvir[halo_id]
        halo_mfof[ihalo]=halo.GroupMFOF[halo_id]

        number[ihalo] = halo.GroupLen[halo_id]
        particle_mass[ihalo] = snap.Particle_mass
        boxsize[ihalo] = snap.BoxSize

        rhalo=halo.GroupR200[halo_id]

        r200[ihalo]=halo.GroupR200[halo_id]
        rvir[ihalo]=halo.GroupRvir[halo_id]
        Ekin[ihalo] = halo.Ekin[halo_id]
        Epot[ihalo] = halo.Epot[halo_id]
        eta[ihalo] = 2 * Ekin[ihalo] / abs(Epot[ihalo]) if Epot[ihalo] != 0 else float('inf')
        angular_momentum[ihalo] = np.linalg.norm([halo.Lx[halo_id], halo.Ly[halo_id], halo.Lz[halo_id]])
        spin[ihalo] = angular_momentum[ihalo] * np.sqrt(np.abs(Ekin[ihalo]+Epot[ihalo])) / (G * halo_m200[ihalo]**2.5)

        c_a[ihalo] = halo.Minor_axis_ratio[halo_id]
        b_a[ihalo] = halo.semi_major_axis_ratio[halo_id]
    
        # Centre of halo
        pos_halo=halo.GroupPos[halo_id]
        pids=kdtree.query_ball_point(pos_halo,r=rhalo)[0]#,return_sorted=True)#,return_length=True)    
        dr=snap.pos[pids]-halo.GroupPos[halo_id]
        dr[:,0]=np.where(dr[:,0]>0.5*snap.BoxSize,dr[:,0]-snap.BoxSize,dr[:,0])
        dr[:,0]=np.where(dr[:,0]<-0.5*snap.BoxSize,dr[:,0]+snap.BoxSize,dr[:,0])
        dr[:,1]=np.where(dr[:,1]>0.5*snap.BoxSize,dr[:,1]-snap.BoxSize,dr[:,1])
        dr[:,1]=np.where(dr[:,1]<-0.5*snap.BoxSize,dr[:,1]+snap.BoxSize,dr[:,1])
        dr[:,2]=np.where(dr[:,2]>0.5*snap.BoxSize,dr[:,2]-snap.BoxSize,dr[:,2])
        dr[:,2]=np.where(dr[:,2]<-0.5*snap.BoxSize,dr[:,2]+snap.BoxSize,dr[:,2])

        r=np.sqrt(dr[:,0]**2+dr[:,1]**2+dr[:,2]**2)
        indx=np.where(r<rhalo)[0]

        xc=np.sum(snap.pos[pids][indx][:,0])/snap.pos[pids][indx][:,0].size
        yc=np.sum(snap.pos[pids][indx][:,1])/snap.pos[pids][indx][:,1].size
        zc=np.sum(snap.pos[pids][indx][:,2])/snap.pos[pids][indx][:,2].size

        d=halo.GroupPos[halo_id]-np.array([xc,yc,zc]).T
        d[:,0]=np.where(d[:,0]>0.5*snap.BoxSize,d[:,0]-snap.BoxSize,d[:,0])
        d[:,0]=np.where(d[:,0]<-0.5*snap.BoxSize,d[:,0]+snap.BoxSize,d[:,0])
        d[:,1]=np.where(d[:,1]>0.5*snap.BoxSize,d[:,1]-snap.BoxSize,d[:,1])
        d[:,1]=np.where(d[:,1]<-0.5*snap.BoxSize,d[:,1]+snap.BoxSize,d[:,1])
        d[:,2]=np.where(d[:,2]>0.5*snap.BoxSize,d[:,2]-snap.BoxSize,d[:,2])
        d[:,2]=np.where(d[:,2]<-0.5*snap.BoxSize,d[:,2]+snap.BoxSize,d[:,2])

        diff=np.sqrt(d[:,0]**2+d[:,1]**2+d[:,2]**2)
        offset[ihalo]=diff/halo.GroupR200[halo_id]

        # velocity dispersion
        xx = halo.xx[halo_id]
        xy = halo.xy[halo_id]
        xz = halo.xz[halo_id]
        yx = halo.yx[halo_id]
        yy = halo.yy[halo_id]
        yz = halo.yz[halo_id]
        zx = halo.zx[halo_id]
        zy = halo.zy[halo_id]
        zz = halo.zz[halo_id]
        inertia_tensor = np.array([[xx, xy, xz],
                                   [yx, yy, yz],
                                   [zx, zy, zz]])
        inertia_tensor = np.squeeze(inertia_tensor)
    
        veldisp_xx = halo.veldisp_xx[halo_id]
        veldisp_xy = halo.veldisp_xy[halo_id]
        veldisp_xz = halo.veldisp_xz[halo_id]
        veldisp_yx = halo.veldisp_yx[halo_id]
        veldisp_yy = halo.veldisp_yy[halo_id]
        veldisp_yz = halo.veldisp_yz[halo_id]
        veldisp_zx = halo.veldisp_zx[halo_id]
        veldisp_zy = halo.veldisp_zy[halo_id]
        veldisp_zz = halo.veldisp_zz[halo_id]
        veldisp_matrix = np.array([[veldisp_xx, veldisp_xy, veldisp_xz],
                                   [veldisp_yx, veldisp_yy, veldisp_yz],
                                   [veldisp_zx, veldisp_zy, veldisp_zz]])
        veldisp_matrix = np.squeeze(veldisp_matrix)
        
        eigenvalues_inertia, _ = eigh(inertia_tensor)
        eigenvalues_veldisp, _ = eigh(veldisp_matrix)
    
        veldisp_3d[ihalo] = np.linalg.norm(np.sqrt(np.abs(eigenvalues_veldisp)))
        veldisp_1d[ihalo] = veldisp_3d[ihalo] / np.sqrt(3)
        
        sorted_indices = np.argsort(np.abs(eigenvalues_veldisp))[::-1]
        veldisp_x = np.sqrt(np.abs(eigenvalues_veldisp[sorted_indices[0]]))
        veldisp_y = np.sqrt(np.abs(eigenvalues_veldisp[sorted_indices[1]]))
        veldisp_z = np.sqrt(np.abs(eigenvalues_veldisp[sorted_indices[2]]))
    
        siga[ihalo] = veldisp_x
        sigb[ihalo] = veldisp_y
        sigc[ihalo] = veldisp_z
        sigv[ihalo] = halo.sigV[halo_id]  

        # properties of subhalos
        Nsub[ihalo] = halo.GroupNsubs[halo_id]
        Vmax[ihalo] = halo.Vmax[halo_id]
        subhalo_ids = np.where(halo_id+1==halo.ParentHaloID)#[0]
        for subhalo_id in subhalo_ids:
            if len(halo.GroupM200[subhalo_id]) > 2:
                m200_sub[ihalo] = np.max(halo.GroupM200[subhalo_id])
                mvir_sub[ihalo] = np.max(halo.GroupMvir[subhalo_id])
                mtot_sub[ihalo] = np.max(halo.GroupMass[subhalo_id])
                mfof_sub[ihalo] = np.max(halo.GroupMFOF[subhalo_id])
                
                Vmax_sub[ihalo] = np.max(halo.Vmax[subhalo_id])
                veldisp_sub[ihalo] = np.linalg.norm(np.std(halo.GroupVel[subhalo_id],axis=0))
                # veldisp_submbp[ihalo] = np.linalg.norm(np.std(halo.GroupVelmbp[subhalo_id],axis=0))
                # veldisp_subminpot[ihalo] = np.linalg.norm(np.std(halo.GroupVelminpot[subhalo_id],axis=0))
            else:
                m200_sub[ihalo] = 0  
                Vmax_sub[ihalo] = 0
                veldisp_sub[ihalo] = 0
                # veldisp_submbp[ihalo] = 0
                # veldisp_subminpot[ihalo] = 0
    
        frac[ihalo] = m200_sub[ihalo]/halo_m200[ihalo]
        frac_v[ihalo] = Vmax_sub[ihalo]/Vmax[ihalo]
        major_merger[ihalo] = find_most_massive_merger(redshift, m200)
        max_mass[ihalo], max_redshift[ihalo] = find_max_mass_redshift(redshift, m200)
        
    ihalo+=1

end_time=perf_counter()
print(f'It took {end_time-start_time:0.2f} seconds (s)')  

It took 111.62 seconds (s)


In [5]:
Properties = np.column_stack((number,halo_m200,halo_masstot,halo_mvir,halo_mfof,r200,rvir,offset,
                              Ekin,Epot,eta,angular_momentum,spin,
                              veldisp_3d,veldisp_1d,siga,sigb,sigc,sigv,c_a,b_a,
                              particle_mass,boxsize,Nsub,m200_sub,mvir_sub,mtot_sub,mfof_sub,
                              veldisp_sub,Vmax,Vmax_sub,
                              frac,frac_v,formation_redshift,alpha03,major_merger,max_mass,max_redshift))

Labels = ['Number_particles', 'M200', 'Mtot','Mvir','MFOF', 'R200', 'Rvir', 'Offset', 
          'kinetic_energy', 'potential_energy', 'energy_ratio',
          'angular_momentum','spin_parameter', 'Sigma3D', 'Sigma1D',' Sigma_major','Sigma_semi','Sigma_minor','Sigmav',
          'axis_ratio_c/a','axis_ratio_b/a',
          'particle_mass', 'box_size','N_subhalos','m200_subhalo','Mvir_subhalo','Mtot_subhalo','Mfof_subhalo',
          'Sigma_subhalos','Vmax','Vmax_subhalo',
          'fraction_subhalo','fraction_v_subhalo','formation_time','accretion_rate','major_merger','max_mass','max_redshift'] 

data = np.vstack([Labels, Properties])

# np.savetxt('./data/L31pt25_N512/L31pt25_N512.txt', data, fmt='%s')
# np.savetxt('./data/L62pt5_N512/L62pt5_N512.txt', data, fmt='%s')
# np.savetxt('./data/L125_N512/L125_N512.txt', data, fmt='%s')
np.savetxt('./data/L250_N512/L250_N512.txt', data, fmt='%s')
# np.savetxt('./data/L500_N512/L500_N512.txt', data, fmt='%s')
# np.savetxt('./data/L1000_N512/L1000_N512.txt', data, fmt='%s')